In [1]:
!pip install deepl

In [2]:
import deepl

deepl_api_key = "60b8242b-323f-499c-afd4-ad5ae855f47c"
translator = deepl.Translator(deepl_api_key)

result = translator.translate_text("Hello world!", target_lang="FR")
print(result.text)

result = translator.translate_text("Hallo Welt!", target_lang="FR")
print(result.text)

Bonjour à tous !
Bonjour le monde !


In [3]:
import pandas as pd

# Lecture des datas
df = pd.read_csv("fichier_nettoye.csv")          # Fichier nettoye traité en P1
df.head()

,Unnamed: 0.1,Unnamed: 0,productid,imageid,prdtypecode,merged,nb_caracteres,langue,merged_tronq,merged_corr
0,0,0,3804725264,1263597046,10,olivia: personalisiertes notizbuch / 150 seite...,90,it,olivia: personalisiertes notizbuch / 150 seite...,oliva personalisiertes notizbuch / 150 siete /...
1,1,1,436067568,1008141237,2280,journal des arts (le) ndeg 133 du 28/09/2001 -...,210,fr,journal des arts (le) ndeg 133 du 28/09/2001 -...,journal des arts (le) ndeg 133 du 28/09/2001 -...
2,2,2,201115110,938777978,50,grand stylet ergonomique bleu gamepad nintendo...,839,fr,grand stylet ergonomique bleu gamepad nintendo...,grand stylet ergonomique bleu gamepad nintendo...
3,3,3,50418756,457047496,1280,peluche donald - europe - disneyland 2000 (mar...,65,fr,peluche donald - europe - disneyland 2000 (mar...,peluche donald - europe - disneyland 2000 (mar...
4,4,4,278535884,1077757786,2705,la guerre des tuques - luc a des idées de gran...,210,fr,la guerre des tuques - luc a des idées de gran...,la guerre des tuques - luc a des idées de gran...


In [4]:
# Nous allons utiliser l'API Deepl afin de traduire les données qui ne sont pas en français.
# Cependant nous sommes limités à un quota de 500 000 caractères chacun, soit 1 500 000 au total.

# On compte le nombre de caractères total
df["nb_caracteres"] = df["merged"].str.len()
print("Il y a ", df["nb_caracteres"].sum(),"caractères")

# Il y a 48 millions de caractères.
# En amont de l'utilisation de Deepl, il va falloir identifier la langue pour traduire uniquement ce qui n'est pas en français
# Il semble que l'identification de la langue via Deepl consomme le quota, nous allons utiliser une alternative gratuite : langdetect


Il y a  48203434 caractères


In [5]:
pip install langdetect

Note: you may need to restart the kernel to use updated packages.


In [6]:
# Identification de la langue via langdetect
from langdetect import detect, DetectorFactory

DetectorFactory.seed = 42       # On fixe le seed pour avoir toujours le même résultat 
                                # En effet, il peut y avoir des résultats variables et aléatoires sur les textes trop courts entre 2 exécutions 

def detect_language(text):
    try:
        return detect(text)
    except:
        return "unknown"

df["langue"] = df["merged"].apply(detect_language)
df.head()


,Unnamed: 0.1,Unnamed: 0,productid,imageid,prdtypecode,merged,nb_caracteres,langue,merged_tronq,merged_corr
0,0,0,3804725264,1263597046,10,olivia: personalisiertes notizbuch / 150 seite...,90,it,olivia: personalisiertes notizbuch / 150 seite...,oliva personalisiertes notizbuch / 150 siete /...
1,1,1,436067568,1008141237,2280,journal des arts (le) ndeg 133 du 28/09/2001 -...,210,fr,journal des arts (le) ndeg 133 du 28/09/2001 -...,journal des arts (le) ndeg 133 du 28/09/2001 -...
2,2,2,201115110,938777978,50,grand stylet ergonomique bleu gamepad nintendo...,839,fr,grand stylet ergonomique bleu gamepad nintendo...,grand stylet ergonomique bleu gamepad nintendo...
3,3,3,50418756,457047496,1280,peluche donald - europe - disneyland 2000 (mar...,65,fr,peluche donald - europe - disneyland 2000 (mar...,peluche donald - europe - disneyland 2000 (mar...
4,4,4,278535884,1077757786,2705,la guerre des tuques - luc a des idées de gran...,210,fr,la guerre des tuques - luc a des idées de gran...,la guerre des tuques - luc a des idées de gran...


In [7]:
# Nombre de caractères pour les textes qui ne sont pas en français
carac_non_fr = df[df["langue"] != "fr"]["nb_caracteres"].sum()
print("Nombre total de caractères à traduire :", carac_non_fr)

# Il reste toujours 3.8 millions qui ne sont pas en français.
# C'est trop pour notre quota Deepl.

Nombre total de caractères à traduire : 3876513


In [8]:
# Nous choisissons de tronquer les lignes avec trop caractères.
df["merged_tronq"] = df["merged"].apply(lambda x: x[:2000] if isinstance(x, str) else x)
carac_non_fr = df[df["langue"] != "fr"]["merged_tronq"].str.len().sum()
print("Nombre total de caractères à traduire :", carac_non_fr)

# Malgré le tronque à 2000 caractères, il reste quasiment autant de caractères à traduire...


Nombre total de caractères à traduire : 3668733


In [9]:
# Nous avons finalement opté pour tronquer à 500 caractères
df["merged_tronq"] = df["merged"].apply(lambda x: x[:500] if isinstance(x, str) else x)
carac_non_fr = df[df["langue"] != "fr"]["merged_tronq"].str.len().sum()
print("Nombre total de caractères à traduire :", carac_non_fr)


Nombre total de caractères à traduire : 2455722


In [10]:
# Maintenant que les langues ont été identifiées, et avant de procéder à la traduction,
# nous pouvons procéder à un traitement de correction orthographique car nous avons identifiés qu'il y avait des mots avec des fautes de frappes.
# Et les mots mals orthographiés peuvent nuire à la bonne traduction du texte

# Nous nous sommes renseigner sur des solutions performantes telles que Jamspell ou LanguageTool,
# Malheureusement ces solutions nécessaires d'avoir des modèles de langue

# Nous avons opté pour une solution plus simple (mais peut être un peu moins performante) : pyspellchecker

%pip install pyspellchecker
from spellchecker import SpellChecker

# Tests
spell = SpellChecker(language='fr')
print(spell.correction("avc"))  
print(spell.correction("crtains"))
print(spell.correction("lvres"))

Note: you may need to restart the kernel to use updated packages.
avec
certains
livres


In [11]:
# Occurences des langues dans le jeu de données
langues_presentes = df['langue'].value_counts()
print(langues_presentes)

langue
fr    65078
en    10386
it     1622
ca     1215
ro      640
es      622
nl      579
da      488
no      382
af      362
de      324
pt      254
sv      203
cy      176
sl      167
id      145
et      133
hr      124
tl      114
pl       69
fi       65
sw       56
so       50
sq       37
sk       37
cs       30
lt       29
lv       26
tr       25
hu       18
vi        7
Name: count, dtype: int64


In [12]:
# Parmi les langues identifiées, voici celles que pyspellchecker sait traiter : 
# Français - 'fr' :
# Anglais - 'en' : 10386 occurences
# Italien - 'it' : 1622 occurences
# Espagnol - 'es' : 622 occurences
# Néerlandais - 'nl' : 579 occurences
# Allemand - 'de' : 324 occurences
# Portuguais - 'pt' : 254 occurences

In [13]:
# Correction orthographique

from spellchecker import SpellChecker

correcteurs = {
    'fr': SpellChecker(language='fr'),
    'en': SpellChecker(language='en'),
    'it': SpellChecker(language='it'),
    'es': SpellChecker(language='es'),
    'nl': SpellChecker(language='nl'),
    'de': SpellChecker(language='de'),
    'pt': SpellChecker(language='pt'),
}


def correction_orthographique(texte, langue):
    if not isinstance(texte, str) or langue not in correcteurs:
        return texte  # On retourne le texte original si langue non supportée
    spell = correcteurs[langue]
    mots = texte.split()
    mots_corrigés = [spell.correction(mot) or mot for mot in mots]
    return " ".join(mots_corrigés)

df['merged_corr'] = df.apply(
    lambda row: correction_orthographique(row['merged_tronq'], row['langue']),
    axis=1
)

In [14]:
# Attention, le code ci-dessus pour la correction orthographique a mis environ 3 jours à s'exécuter.

df.to_csv("fichier_corr.csv", index=False)
df.to_excel("fichier_corr.xlsx", index=False)
df.head()

,Unnamed: 0.1,Unnamed: 0,productid,imageid,prdtypecode,merged,nb_caracteres,langue,merged_tronq,merged_corr
0,0,0,3804725264,1263597046,10,olivia: personalisiertes notizbuch / 150 seite...,90,it,olivia: personalisiertes notizbuch / 150 seite...,oliva personalisiertes notizbuch / 150 siete /...
1,1,1,436067568,1008141237,2280,journal des arts (le) ndeg 133 du 28/09/2001 -...,210,fr,journal des arts (le) ndeg 133 du 28/09/2001 -...,journal des arts le de 133 du 28/09/2001 - par...
2,2,2,201115110,938777978,50,grand stylet ergonomique bleu gamepad nintendo...,839,fr,grand stylet ergonomique bleu gamepad nintendo...,grand stylet ergonomique bleu gamepad nintendo...
3,3,3,50418756,457047496,1280,peluche donald - europe - disneyland 2000 (mar...,65,fr,peluche donald - europe - disneyland 2000 (mar...,peluche tonal - euros - disneyland 2000 marion...
4,4,4,278535884,1077757786,2705,la guerre des tuques - luc a des idées de gran...,210,fr,la guerre des tuques - luc a des idées de gran...,la guerre des tiques - lui a des idées de gran...


In [15]:
# Fonction pour n'envoyer à l'API Deepl que ce qui n'est pas en français
def translate_text(text, lang):
    if lang != "fr":  # Si la langue n'est pas français, on traduit
        try:
            return translator.translate_text(text, target_lang="FR").text
        except Exception as e:
            print(f"Erreur de traduction : {e}")
            return text  # En cas d'erreur, on retourne le texte original
    return text  # Si déjà en français, on retourne également le texte original


In [16]:
# Traduction de la partie tronquée uniquement pour les lignes qui ne sont pas déjà en français
df["merged_tronq_trad"] = df.apply(lambda x: translate_text(x["merged_tronq"], x["langue"]), axis=1)

# Sauvegarde
df.to_csv("fichier_traduit.csv", index=False)
df.to_excel("fichier_traduit.xlsx", index=False)
